# Golem Bootcamp 2022 - Klasyfikacja
*22.11.2022*

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Dane


In [ ]:
batch_size_train = 64
batch_size_test = 1000

In [ ]:
train_loader = torch.utils.data.DataLoader(
  datasets.FashionMNIST('./data/', train=True, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  datasets.FashionMNIST('./data/', train=False, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape

Tłumaczymy co oznaczają poszczególne klasy

In [ ]:
label_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boo'
}

Wyświetlamy przykładowe punkty danych

In [ ]:
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(label_dict[example_targets[i].item()]))
  plt.xticks([])
  plt.yticks([])


## Model

In [ ]:
class ShallowNet(nn.Module):
    def __init__(self):
        super(ShallowNet, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        return F.log_softmax(x)

### Neuron

... z biologii

![Biological Neuron](./img/biological_network.png)

... z informatyki

![Neuron](./img/neuron.png)

### Sztuczna sieć neuronowa

![Artificial Neural Network](./img/neural_network.png)

## Hiperparametry

**batch mode**:   
Where the batch size is equal to the total dataset thus making the iteration and epoch values equivalent  

**mini-batch mode**:  
Where the batch size is greater than one but less than the total dataset size. Usually, a number that can be divided into the total dataset size  

**stochastic mode**:   
Where the batch size is equal to one. Therefore the gradient and the neural network parameters are updated after each sample.  

In [ ]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 420
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
network = ShallowNet()
optimizer = optim.SGD(
    network.parameters(), 
    lr=learning_rate,
    momentum=momentum
    )


## Trening

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './models/model.pth')
      torch.save(optimizer.state_dict(), './models/optimizer.pth')

In [ ]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
test()

In [ ]:
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

## Poprawa wyniku

Wynik wyszedł taki sobie - jak można go poprawić?

* dłuższy czas treningu (więcej `epoch`)
* większy model (więcej warstw w klasie `ShallowNet`)
* zmiana `optimizera` np. ADAM
* zmiana `batch_size`
* zmiana `learning_rate`

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

## Further reading

[A.Karpathy blog post](https://karpathy.github.io/2019/04/25/recipe/)  
[A.Karpathy course (in progress)](https://github.com/karpathy/nn-zero-to-hero)  

[StatQuest series on NN](https://www.youtube.com/playlist?list=PLblh5JKOoLUIxGDQs4LFFD--41Vzf-ME1)  
[3Blue1Brown series on NN](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)  

# THE END